In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import Span
from shapely.geometry import Point
import geopandas as gpd
import glob
from datetime import datetime
import bokeh
from bokeh.layouts import Row, column, gridplot
from bokeh.models import Title, Legend, TapTool, Range1d, TabPanel, Tabs
import matplotlib as mpl

from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)

{Warning(code=1000, name='MISSING_RENDERERS', description='Plot has no renderers'),
 Warning(code=1002, name='EMPTY_LAYOUT', description='Layout has no children')}

In [2]:
color_palette = [  '#4E79A7',  # Blue
    '#F28E2B',  # Orange
    '#E15759',  # Red
    '#76B7B2',  # Teal
    '#59A14F',  # Green
    '#EDC948',  # Yellow
    '#B07AA1',  # Purple
    '#FF9DA7',  # Pink
    '#9C755F',  # Brown
    '#BAB0AC',  # Gray
    '#7C7C7C',  # Dark gray
    '#6B4C9A',  # Violet
    '#D55E00',  # Orange-red
    '#CC61B0',  # Magenta
    '#0072B2',  # Bright blue
    '#329262',  # Peacock green
    '#9E5B5A',  # Brick red
    '#636363',  # Medium gray
    '#CD9C00',  # Gold
    '#5D69B1',  # Medium blue
]

# Meta Mobile Network Connectivity

The Network coverage maps, as described in the previous page, show the number of points (~600m) that are likely to **not** have mobile network coverage compared to a 30 day prior baseline. 

The first step here is to download the data from the Facebook Data For Good Portal. One member in your team/members of the Data Lab team will be able to pass those files to you. 

Following this, the latitude and logintude points are used to categorize the data at different admin levels

In [3]:
all_files = glob.glob('../../data/meta-network-connectivity/network_undetected/*.csv')

networkCoverage = pd.DataFrame()

li = []

for file in all_files:
    #print(file.split('_')[3])
    df1 = pd.read_csv(file)
    #print(file.split('_')[2])
    df1['date'] = datetime.strptime(file.split('_')[2], '%Y-%m-%d')
    li.append(df1)

networkCoverage = pd.concat(li, axis=0)
networkCoverage = networkCoverage[networkCoverage['country']=='TR']

In [4]:
# read turkey shapefile and areas of interest
turkey_adm2 = gpd.read_file('../../data/turkey_administrativelevels0_1_2/tur_polbna_adm2.shp')
aoi = gpd.read_file('../../data/SYRTUR_tessellation.geojson')
affected_adm1 = list(turkey_adm2[turkey_adm2['adm1'].isin(aoi['ADM1_PCODE'].unique())]['adm1_en'].unique())
affected_adm2 = list(turkey_adm2[turkey_adm2['pcode'].isin(aoi['ADM2_PCODE'].unique())]['adm2_en'].unique())

In [5]:
# filter and clean the dataset
def clean_df(networkCoverage, date=None):
    if date is not None:
        df = networkCoverage[networkCoverage['date']==date]
    else:
        df = networkCoverage
    df = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.lon, df.lat))
    df = df.set_crs(4326, allow_override=True)
    df = turkey_adm2.sjoin(df, how = 'left')
    df = df.groupby(['adm2_en', 'date']).sum('no_coverage').reset_index()[['adm2_en', 'date', 'no_coverage']]
    df = df.merge(turkey_adm2[['adm2_en', 'adm1_en']], on = ['adm2_en'], how = 'right').sort_values(by=['adm2_en', 'date']).reset_index(drop=True)

    return df

In [6]:
networkCoverage = clean_df(networkCoverage)
#calculate sum of values, grouped by week
week = networkCoverage.groupby([pd.Grouper(key='date', freq='W-MON'), 'adm2_en', 'adm1_en']).mean().reset_index()

In [7]:
aoi = gpd.read_file('../../data/SYRTUR_tessellation.geojson')['ADM2_PCODE'].unique()
aoi_adm1 = turkey_adm2[turkey_adm2['pcode'].isin(aoi)]['adm1_en'].unique()

In [8]:
bokeh.core.validation.silence(EMPTY_LAYOUT, True)

def get_line_plot(networkCoverage,title, source, earthquakes=False, subtitle=None):

    p2 = figure(x_axis_type = 'datetime', width = 800, height = 400, toolbar_location='above')
    p2.add_layout(Legend(), "right")

    for id, adm2 in enumerate(networkCoverage['adm2_en'].unique()):
        df = networkCoverage[networkCoverage['adm2_en']==adm2][['date', 'no_coverage']].reset_index(drop=True)
        p2.line(df['date'], df['no_coverage'], line_width=2, line_color = color_palette[id], legend_label=adm2)

    p2.legend.click_policy='hide'
    if subtitle is not None:
        p2.title = subtitle


    title_fig = figure(title=title, toolbar_location=None,width=800, height=40, )
    title_fig.title.align = "left"
    title_fig.title.text_font_size = "20pt"
    title_fig.border_fill_alpha = 0
    title_fig.outline_line_width=0

    #with silence(MISSING_RENDERERS):
    sub_title = figure(title=source, toolbar_location=None,width=800, height=40, )
    sub_title.title.align = "left"
    sub_title.title.text_font_size = "10pt"
    sub_title.title.text_font_style="normal"
    sub_title.border_fill_alpha = 0
    sub_title.outline_line_width=0

    layout = column(title_fig, p2, sub_title)

    if earthquakes:
        p2.renderers.extend([
        Span(
            location=datetime(2023, 2, 6),
            dimension="height",
            line_color='#7C7C7C',
            line_width=2,
            line_dash=(4,4)
      ),
        Span(
            location=datetime(2023, 2, 20),
            dimension="height",
            line_color='#7C7C7C',
            line_width=2,
            line_dash=(4,4)
        ),
    ]
)

    return layout



In [9]:
output_notebook()

tabs = []

for adm in aoi_adm1:
    df = networkCoverage[networkCoverage['adm1_en']==adm] 

    tabs.append(
        TabPanel(
        child=get_line_plot(df, f"Areas with Undetected Network post earthquake", "Source: Data for Good Meta", earthquakes=True, subtitle = '~600m areas without network post earthquake compared to 30 day prior baseline'),
                    title=adm.capitalize(),
                )
                )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

Loading BokehJS ...

In [10]:
output_notebook()


tabs = []

for adm in aoi_adm1:
    df = week[week['adm1_en']==adm] 

    tabs.append(
        TabPanel(
        child=get_line_plot(df, f"Weekly trends for Areas with Undetected Network post earthquake", "Source: Data for Good Meta", earthquakes=True, subtitle = '~600m areas without network post earthquake compared to 30 day prior baseline'),
                    title=adm.capitalize(),
                )
                )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

Loading BokehJS ...